In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,ExtraTreesRegressor , RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle

C:\Users\master\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\master\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\master\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
df = pd.read_csv('train.csv', delimiter=',')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
df['LotFrontage'].value_counts(dropna=False)

NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 111, dtype: int64

In [4]:
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df['LotFrontage'].value_counts(dropna=False)

70.049958     259
60.000000     143
70.000000      70
80.000000      69
50.000000      57
             ... 
106.000000      1
38.000000       1
140.000000      1
138.000000      1
137.000000      1
Name: LotFrontage, Length: 111, dtype: int64

In [5]:
df['MasVnrArea'].value_counts(dropna=False)
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

In [6]:
#df['Electrical'].value_counts()
df['Electrical'] = df['Electrical'].fillna('SBrkr')

In [7]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
df['GarageYrBlt'].value_counts(dropna=False)

0.0       81
2005.0    65
2006.0    59
2004.0    53
2003.0    50
          ..
1906.0     1
1927.0     1
1900.0     1
1908.0     1
1933.0     1
Name: GarageYrBlt, Length: 98, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [15]:
df = pd.get_dummies(df)
y = df['SalePrice'].values
X = df[:].values

In [ ]:
X = df.iloc[:, :-1].values
X

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [17]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [18]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[200624. 200624.]
 [133000. 133000.]
 [110000. 110000.]
 [192000. 192000.]
 [ 88000.  88000.]
 [ 85000.  85000.]
 [282922. 282922.]
 [141000. 141000.]
 [745000. 745000.]
 [148800. 148800.]
 [208900. 208900.]
 [136905. 136905.]
 [225000. 225000.]
 [123000. 123000.]
 [119200. 119200.]
 [145000. 145000.]
 [190000. 190000.]
 [123600. 123600.]
 [149350. 149350.]
 [155000. 155000.]
 [166000. 166000.]
 [144500. 144500.]
 [110000. 110000.]
 [174000. 174000.]
 [185000. 185000.]
 [168000. 168000.]
 [177500. 177500.]
 [ 84500.  84500.]
 [320000. 320000.]
 [118500. 118500.]
 [110000. 110000.]
 [213000. 213000.]
 [156000. 156000.]
 [250000. 250000.]
 [372500. 372500.]
 [175000. 175000.]
 [277500. 277500.]
 [112500. 112500.]
 [263000. 263000.]
 [325000. 325000.]
 [243000. 243000.]
 [130000. 130000.]
 [164990. 164990.]
 [280000. 280000.]
 [403000. 403000.]
 [119000. 119000.]
 [125000. 125000.]
 [128200. 128200.]
 [172500. 172500.]
 [ 84900.  84900.]
 [412500. 412500.]
 [156000. 156000.]
 [167900. 16

In [20]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

4.44474737476165e-21

In [24]:
filename = 'Regression_model.sav'
pickle.dump(regressor, open(filename, 'wb'))